In [1]:
########################################################################
# File       : Read_and_Display_Images_in_Napari.ipynb
# Version    : 0.4
# Author     : czsrh
# Date       : 16.05.2021
# Institution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
########################################################################

***Import Libaries to run this notebook***

* [PyPi - czifile](https://pypi.org/project/czifile/)

Thanks to Christoph Gohlke. For details and the source code please see [czifile.py](https://www.lfd.uci.edu/~gohlke/code/czifile.py.html)

* [PyPi - AICSImageIO](https://pypi.org/project/aicsimageio/)

Library to read OME-TIFF and CZI images. It uses the powerful [libCZI](https://github.com/zeiss-microscopy/libCZI) C++ library to read CZI images.

* [PyPi - napari](https://pypi.org/project/napari/)

Very new, fast and cool multi-dimensional array viewer. For more details see the prject page [GitHub - Napari](https://github.com/napari/napari)

***

Another important modules

* [PyPi - xmltodict](https://pypi.org/project/xmltodict/)

This is turns an XML, e.g. the OME-XML or CZI metadata into an "easy-to-use" python dictionary.

In [1]:
# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import sys
import tools.imgfile_tools as imf
import tools.napari_tools as nap

# import the libraries mentioned above
import czifile as zis
from aicsimageio import AICSImage, imread, imread_dask
import xmltodict
import os
import numpy as np
import napari
import xml.etree.ElementTree as ET
from IPython.display import display, HTML

In [13]:
# define your testfiles here

#filename = r"C:\Users\m1srh\OneDrive - Carl Zeiss AG\Smart_Microscopy_Workshop\datasets\celldivision\CellDivision_T=3_Z=5_CH=2_X=240_X=170.ome.tiff"
#filename = r"C:\Users\m1srh\OneDrive - Carl Zeiss AG\Smart_Microscopy_Workshop\datasets\celldivision\CellDivision_T=15_Z=20_CH=2_DCV.czi"
filename = r"C:\Testdata_Zeiss\LatticeLightSheet\LS_Mitosis_T=150-300.czi"

In [14]:
# !!! Only use for CZI files !!! 

# To read the CZI metadata one can use the czifile functionality

# get czi object using the filename
czi = zis.CziFile(filename)

# get the metadata and close the czi file object
mdczi = czi.metadata()
czi.close()

# parse the XML into a dictionary
metadatadict_czi = xmltodict.parse(mdczi)

In [16]:
# !!! Only use for CZI files !!! 

# Sometimes it can be very usefule to save the CZI metadate as an actual XML file.
# This can be done quite easily by converting the metadata string into an XML tree.
# This tree object can be directly as an XML file

# define the new filename for the XML to be created later
xmlfile = filename.replace('.czi', '_CZI_MetaData.xml')

# get the element tree
tree = ET.ElementTree(ET.fromstring(mdczi))

# write xml to disk
tree.write(xmlfile, encoding='utf-8', method='xml')

print('Write special CZI XML metainformation for: ', xmlfile)

Write special CZI XML metainformation for:  C:\Testdata_Zeiss\LatticeLightSheet\LS_Mitosis_T=150-300_CZI_MetaData.xml


In [17]:
# !!! Only use for CZI files !!! 

# or much shorter
xmlczi = imf.writexml_czi(filename)
print(xmlczi)

C:\Testdata_Zeiss\LatticeLightSheet\LS_Mitosis_T=150-300_CZI_MetaData.xml


### Remark

As one can clearly see there are a lot of metadata not all of them are needed for every workflow.

In [18]:
# get the array and the metadata
metadata, add_metadata = imf.get_metadata(filename)
img = AICSImage(filename)
array = img.dask_data
#array = img.get_image_data()

Detected Image Type (based on extension): czi
CZI is Mosaic : False
CZI is RGB : False
Channel shortname not found : 'ShortName'
Channel shortname not found : 'ShortName'
No M Dimension : 'SizeM'
No B Dimension : 'SizeB'
No S Dimension : 'SizeS'
No H Dimension : 'SizeH'
No I Dimension : 'SizeI'
No V Dimension : 'SizeV'
No Tubelens Mag. : 'TubeLenses' Using Default Value = 1.0.
No Tubelens Mag. : 'TubeLenses' Using Default Value = 1.0.
No Objective Name : 'Name'
No Objective Immersion : 'Immersion'
No Objective NA : 'LensNA'
No Tubelens Mag. : 'TubeLenses' Using Default Value = 1.0.
No Nominal Mag. : 'NominalMagnification' Using Default Value = 1.0.
DetectorType not found : 'Type'
Trying to extract Scene and Well information if existing ...
No valid Scene or Well information found: 'S'
Scales will not be rounded.


In [19]:
# shape of numpy array
print('Array Shape                   : ', array.shape)

# shape and dimension entry from CZI file as returned by czifile.py
print('CZI Array Shape (czifile)     : ', metadata['shape_czifile'])
print('CZI Dimension Entry (czifile) : ', metadata['axes_czifile'])
print('Array Shape (AICSImageIO)     : ', metadata['Shape_aics'])
print('Dimension Entry (AICSImageIO) : ', metadata['Axes_aics'])

Array Shape                   :  (1, 151, 2, 70, 413, 434)
CZI Array Shape (czifile)     :  (151, 2, 70, 413, 434, 1)
CZI Dimension Entry (czifile) :  TCZYX0
Array Shape (AICSImageIO)     :  (1, 151, 2, 70, 413, 434)
Dimension Entry (AICSImageIO) :  STCZYX


In [20]:
# show dimensions and scaling

print('SizeT : ', metadata['SizeT'])
print('SizeZ : ', metadata['SizeZ'])
print('SizeC : ', metadata['SizeC'])
print('SizeX : ', metadata['SizeX'])
print('SizeY : ', metadata['SizeY'])
print('XScale: ', metadata['XScale'])
print('YScale: ', metadata['YScale'])
print('ZScale: ', metadata['ZScale'])

SizeT :  151
SizeZ :  70
SizeC :  2
SizeX :  434
SizeY :  413
XScale:  0.145
YScale:  0.145
ZScale:  0.145


In [21]:
### show complete metadata

# convert metadata dictionary to a pandas dataframe
mdframe = imf.md2dataframe(metadata)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(mdframe.to_html()))

,Parameter,Value
0,Directory,C:\Testdata_Zeiss\LatticeLightSheet
1,Filename,LS_Mitosis_T=150-300.czi
2,Extension,czi
3,ImageType,czi
4,AcqDate,2019-11-23T13:27:18.761492Z
5,TotalSeries,None
6,SizeX,434
7,SizeY,413
8,SizeZ,70
9,SizeC,2


In [22]:
import time
viewer = napari.Viewer()
time.sleep(3)

# configure napari automatically based on metadata and show stack
layers = nap.show_napari(viewer, array, metadata,
                                 blending='additive',
                                 adjust_contrast=False,
                                 gamma=0.85,
                                 add_mdtable=True,
                                 rename_sliders=True)

Adding Channel  : Dye1
Shape Channel   : 0 (1, 151, 1, 70, 413, 434)
Scaling Factors : [1.0, 1.0, 1.0, 1.0, 1.0]
Adding Channel  : Dye2
Shape Channel   : 1 (1, 151, 1, 70, 413, 434)
Scaling Factors : [1.0, 1.0, 1.0, 1.0, 1.0]
Renaming the Sliders based on the Dimension String ....
No H Dimension found
No V Dimension found
No M Dimension found


jupyter nbconvert Read_and_Display_Images_in_Napari.ipynb --to slides --post serve